In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time
import matplotlib.dates as mdates
import seaborn as sns
import matplotlib.pyplot as plt
from pandasql import *
import seaborn as sns
import warnings
from numpy import *
from sklearn.metrics import accuracy_score
import math
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from math import sqrt
warnings.filterwarnings('ignore')
%matplotlib inline

In [11]:
n = 1000
k = 64
X = np.zeros(shape=(n,k))
np.random.seed(39)
for i in range(n):
    for j in range(k):
        X[i][j] = np.random.uniform()

In [12]:
X=np.array(X)

In [13]:
print(X)

[[0.54688916 0.79789902 0.82040188 ... 0.89582475 0.88657711 0.01363686]
 [0.86886947 0.30145564 0.94794894 ... 0.17431285 0.56729794 0.98401691]
 [0.52914654 0.43819623 0.91374443 ... 0.27539889 0.67386232 0.79933576]
 ...
 [0.4090994  0.20901292 0.4140183  ... 0.28330777 0.75779136 0.45883172]
 [0.81094791 0.40118719 0.5222902  ... 0.97156403 0.66615932 0.41582605]
 [0.05375513 0.60521764 0.27326501 ... 0.89190722 0.99042922 0.85547152]]


In [14]:
def cal_top1(X):
    #top-1 item
    SMALL = -9999.99
    first_best_index = np.zeros(n,dtype=int)
    first_best_val = np.zeros(n,dtype=float)
    for i in range(n):
        first_best_val[i] = SMALL
        first_best_index[i] = -1
        for j in range(n):
            if(i==j):
                continue
            tmp = 0.0
            magnitude1 = 0
            magnitude2 = 0
            for t in range(k):
                tmp += X[i,t] * X[j,t]
                magnitude1 += X[i,t]*X[i,t]
                magnitude2 += X[j,t]*X[j,t]
            tmp/= math.sqrt(magnitude1* magnitude2)
            if(tmp>first_best_val[i]):
                first_best_val[i] = tmp
                first_best_index[i] = j
    return first_best_val,first_best_index

In [15]:
def cal_top1_optimised(X):
    #top-1 item
    SMALL = -9999.99
    first_best_index = np.zeros(n,dtype=int)
    first_best_val = np.zeros(n,dtype=float)
    for i in range(n):
        first_best_val[i] = SMALL
        first_best_index[i] = -1
        for j in range(n):
            if(i==j):
                continue
            tmp = np.dot(X[i],X[j])/(np.sqrt(np.sum(np.square(X[i]))) * np.sqrt(np.sum(np.square(X[j]))))
            if(tmp>first_best_val[i]):
                first_best_val[i] = tmp
                first_best_index[i] = j
    return first_best_val,first_best_index

In [16]:
%time first_best_val,first_best_index = cal_top1(X)

CPU times: user 1min 54s, sys: 146 ms, total: 1min 54s
Wall time: 1min 54s


In [17]:
first_best_val[0:10]

array([0.8690864 , 0.85467417, 0.88301265, 0.84900181, 0.85267185,
       0.8621937 , 0.87595297, 0.86882657, 0.85891488, 0.84937903])

In [18]:
first_best_index[0:10]

array([569,  56, 608, 717, 273, 268, 923, 133, 736, 691])

In [19]:
%time first_best_val,first_best_index = cal_top1_optimised(X)

CPU times: user 18 s, sys: 177 ms, total: 18.2 s
Wall time: 18 s


In [20]:
first_best_val[0:10]

array([0.8690864 , 0.85467417, 0.88301265, 0.84900181, 0.85267185,
       0.8621937 , 0.87595297, 0.86882657, 0.85891488, 0.84937903])

In [21]:
first_best_index[0:10]

array([569,  56, 608, 717, 273, 268, 923, 133, 736, 691])

In [43]:
def cal_top3(X):
    #top-1 item
    SMALL = -9999.99
    first_best_index = np.zeros(n,dtype=int)
    first_best_val = np.zeros(n,dtype=float)
    second_best_index = np.zeros(n,dtype=int)
    second_best_val = np.zeros(n,dtype=float)
    third_best_index = np.zeros(n,dtype=int)
    third_best_val = np.zeros(n,dtype=float)
    for i in range(n):
        first_best_val[i] = SMALL
        first_best_index[i] = -1
        second_best_val[i] = SMALL
        second_best_index[i] = -1
        third_best_val[i] = SMALL
        third_best_index[i] = -1
        for j in range(n):
            if(i==j):
                continue
            tmp = 0.0
            magnitude1 = 0
            magnitude2 = 0
            for t in range(k):
                tmp += X[i,t] * X[j,t]
                magnitude1 += X[i,t]*X[i,t]
                magnitude2 += X[j,t]*X[j,t]
            tmp/= math.sqrt(magnitude1* magnitude2)
            if(tmp>=first_best_val[i]):
                third_best_val[i] = second_best_val[i]
                third_best_index[i] = second_best_index[i]
                second_best_val[i] = first_best_val[i]
                second_best_index[i] = first_best_index[i]
                first_best_val[i] = tmp
                first_best_index[i] = j
            elif(tmp>=second_best_val[i]):
                third_best_val[i] = second_best_val[i]
                third_best_index[i] = second_best_index[i]
                second_best_val[i] = tmp
                second_best_index[i] = j
            elif(tmp>third_best_val[i]):
                third_best_val[i] = tmp
                third_best_index[i] = j
    return first_best_val,first_best_index,second_best_val,second_best_index,third_best_val,third_best_index

In [44]:
def cal_top3_optimised(X):
    #top-1 item
    SMALL = -9999.99
    first_best_index = np.zeros(n,dtype=int)
    first_best_val = np.zeros(n,dtype=float)
    second_best_index = np.zeros(n,dtype=int)
    second_best_val = np.zeros(n,dtype=float)
    third_best_index = np.zeros(n,dtype=int)
    third_best_val = np.zeros(n,dtype=float)
    for i in range(n):
        first_best_val[i] = SMALL
        first_best_index[i] = -1
        second_best_val[i] = SMALL
        second_best_index[i] = -1
        third_best_val[i] = SMALL
        third_best_index[i] = -1
        for j in range(n):
            if(i==j):
                continue
            tmp = np.dot(X[i],X[j])/(np.sqrt(np.sum(np.square(X[i]))) * np.sqrt(np.sum(np.square(X[j]))))
            if(tmp>=first_best_val[i]):
                third_best_val[i] = second_best_val[i]
                third_best_index[i] = second_best_index[i]
                second_best_val[i] = first_best_val[i]
                second_best_index[i] = first_best_index[i]
                first_best_val[i] = tmp
                first_best_index[i] = j
            elif(tmp>=second_best_val[i]):
                third_best_val[i] = second_best_val[i]
                third_best_index[i] = second_best_index[i]
                second_best_val[i] = tmp
                second_best_index[i] = j
            elif(tmp>third_best_val[i]):
                third_best_val[i] = tmp
                third_best_index[i] = j
    return first_best_val,first_best_index,second_best_val,second_best_index,third_best_val,third_best_index

In [142]:
%time first_best_val,first_best_index,second_best_val,second_best_index,third_best_val,third_best_index = cal(X)

CPU times: user 28min 35s, sys: 10.5 s, total: 28min 46s
Wall time: 28min 42s


In [28]:
import gensim

# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/a0m00vn/aps/TathaAstu Hackathon/Finals/Datathon/GoogleNews-vectors-negative300.bin', binary=True)



In [30]:
model['tea']

array([-2.08984375e-01,  1.30859375e-01, -4.95605469e-02,  3.32031250e-01,
        8.15429688e-02,  1.09863281e-01,  6.07910156e-02, -2.06054688e-01,
       -1.31835938e-01,  1.17675781e-01,  1.00097656e-01, -3.67187500e-01,
        2.12890625e-01,  2.92968750e-01, -2.92968750e-01,  1.39160156e-02,
       -1.16699219e-01,  1.58203125e-01,  1.37695312e-01, -1.93359375e-01,
       -2.00195312e-01,  1.12304688e-01,  2.86865234e-02, -2.31445312e-01,
       -6.44531250e-02,  5.46875000e-01, -3.61328125e-01,  7.27539062e-02,
        1.69921875e-01, -7.71484375e-02,  5.63964844e-02, -2.04101562e-01,
       -2.20703125e-01,  2.31933594e-02, -3.00781250e-01, -1.51367188e-01,
       -2.47070312e-01, -3.26171875e-01,  1.99890137e-03, -5.05371094e-02,
       -1.03027344e-01, -1.88476562e-01,  3.33984375e-01, -9.27734375e-03,
       -2.63671875e-01, -1.65039062e-01, -4.00390625e-01, -1.34765625e-01,
        6.83593750e-02,  1.06811523e-02, -1.66015625e-01,  1.72851562e-01,
       -3.51562500e-02, -

In [31]:
model.get_vector('tea')

array([-2.08984375e-01,  1.30859375e-01, -4.95605469e-02,  3.32031250e-01,
        8.15429688e-02,  1.09863281e-01,  6.07910156e-02, -2.06054688e-01,
       -1.31835938e-01,  1.17675781e-01,  1.00097656e-01, -3.67187500e-01,
        2.12890625e-01,  2.92968750e-01, -2.92968750e-01,  1.39160156e-02,
       -1.16699219e-01,  1.58203125e-01,  1.37695312e-01, -1.93359375e-01,
       -2.00195312e-01,  1.12304688e-01,  2.86865234e-02, -2.31445312e-01,
       -6.44531250e-02,  5.46875000e-01, -3.61328125e-01,  7.27539062e-02,
        1.69921875e-01, -7.71484375e-02,  5.63964844e-02, -2.04101562e-01,
       -2.20703125e-01,  2.31933594e-02, -3.00781250e-01, -1.51367188e-01,
       -2.47070312e-01, -3.26171875e-01,  1.99890137e-03, -5.05371094e-02,
       -1.03027344e-01, -1.88476562e-01,  3.33984375e-01, -9.27734375e-03,
       -2.63671875e-01, -1.65039062e-01, -4.00390625e-01, -1.34765625e-01,
        6.83593750e-02,  1.06811523e-02, -1.66015625e-01,  1.72851562e-01,
       -3.51562500e-02, -

In [35]:
model.most_similar('tea',topn = 5)

[('Tea', 0.7009037733078003),
 ('teas', 0.6727380752563477),
 ('shape_Angius', 0.6323482990264893),
 ('activist_Jamie_Radtke', 0.58638596534729),
 ('decaffeinated_brew', 0.5839536190032959)]

In [37]:
model.most_similar('apple',topn = 10)

[('apples', 0.7203598022460938),
 ('pear', 0.6450696587562561),
 ('fruit', 0.6410146355628967),
 ('berry', 0.6302294731140137),
 ('pears', 0.6133961081504822),
 ('strawberry', 0.6058261394500732),
 ('peach', 0.6025873422622681),
 ('potato', 0.596093475818634),
 ('grape', 0.5935864448547363),
 ('blueberry', 0.5866668224334717)]